<a href="https://colab.research.google.com/github/maldoroty/DS5220_project/blob/main/PageRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PageRank

In this assignment, you will compute PageRank on a collection of 469,235 web sites using the iterative version of the PageRank algorithm described in class for sparse graphs.  As we discussed in class, this avoids performing cubic-time operations on a very large matrix.

Furthermore, you should take advantage of the _sparsity_ of the original hyperlinks, even though "teleportation" or random jumps connect every note to every other node.  As long as the maximum degree of the original link graph is much less than the number of nodes, you should be able to keep each iteration's runtime less than quadratic in the number of nodes.

Consider the following directed graph:

![A directed link graph](https://ccs.neu.edu/home/dasmith/courses/cs6200/pagerank.jpg)

We can represent this graph as a collection of nodes, here, ordered pairs of node index and node name:

In [1]:
small_nodes = [(0, 'A'),
              (1, 'B'),
              (2, 'C'),
              (3, 'D'),
              (4, 'E'),
              (5, 'F')]

and a collection of directed links, i.e., ordered pairs from source to target:

In [2]:
small_edges = [
  (0, 1),
  (0, 2),
  (0, 5),
  (1, 2),
  (1, 3),
  (1, 4),
  (1, 5),
  (2, 3),
  (2, 4),
  (3, 0),
  (3, 2),
  (3, 4),
  (3, 5),
  (4, 0),
  (5, 0),
  (5, 1),
  (5, 4)
]

We use integer identifiers for the nodes for efficiency. In most real-world collections of hyperlinks, unlike this example, not every page will have in-links, nor will every page have out-links.

## First Implementation and Test

\[10 points\] Implement the iterative PageRank algorithm we discussed in class. Test your code on the six-node example using the input representation given above.  Be sure that your code handles pages that have no in-links or out-links properly.  (You may wish to test on a few such examples.) In later parts of this assignment, depending on how you store the data, it may be convenient to use iterators rather than storing the data in memory.

In [3]:
import numpy as np

In [4]:
# TODO: Implement PageRank, given nodes and edges, to start with a uniform
# distribution over nodes, run a fixed number of iterations, and
# return a distribution over nodes.
def page_rank_fixed_iter(nodes, edges, iterations=10):
  N = len(nodes)
  d = 0.9
  adj = np.zeros((N, N))
  r = np.ones(N) / N
  for edge in edges:
    i = edge[0]
    j = edge[1]
    adj[j][i] = 1
  adj *= d
  adj += (1 - d) / N
  adj /= adj.sum(axis=0)
  for _ in range(iterations):
    np.matmul(adj, r, out=r)
  return [(nodes[i][0], nodes[i][1], r[i]) for i in range(len(r))]

# Output PageRank on the toy graph at various points.
# Make sure your output has node number, name, and PageRank value.
print(page_rank_fixed_iter(small_nodes, small_edges, 1))
print(page_rank_fixed_iter(small_nodes, small_edges, 10))
print(page_rank_fixed_iter(small_nodes, small_edges, 100))

[(0, 'A', 0.25183736367946896), (1, 'B', 0.11486825171035699), (2, 'C', 0.14237790422000948), (3, 'D', 0.127213303529093), (4, 'E', 0.2213252726410621), (5, 'F', 0.14237790422000948)]
[(0, 'A', 0.2548918935679543), (1, 'B', 0.13872627256156275), (2, 'C', 0.1513772469247119), (3, 'D', 0.11366689408949376), (4, 'E', 0.18996044593156633), (5, 'F', 0.1513772469247119)]
[(0, 'A', 0.2551414922419215), (1, 'B', 0.13872577432226343), (2, 'C', 0.15141287025181044), (3, 'D', 0.11351357955417946), (4, 'E', 0.18979341337802708), (5, 'F', 0.15141287025181044)]


In [5]:
nodes = [(0, 'A'), (1, 'B'), (2, 'C')]
edges = [(0, 1), (1, 2)]

print(page_rank_fixed_iter(nodes, edges, 1))
print(page_rank_fixed_iter(nodes, edges, 10))
print(page_rank_fixed_iter(nodes, edges, 100))

[(0, 'A', 0.1333333333333333), (1, 'B', 0.4333333333333333), (2, 'C', 0.43333333333333335)]
[(0, 'A', 0.17803312243333327), (1, 'B', 0.33872882923333325), (2, 'C', 0.4832380483333332)]
[(0, 'A', 0.17825311942958982), (1, 'B', 0.33868092691622065), (2, 'C', 0.4830659536541884)]


## PageRank on Web Crawl Data

\[20 points\] Download and unpack a list of `.edu` websites and the links among them from the [Common Crawl](https://commoncrawl.org/2017/05/hostgraph-2017-feb-mar-apr-crawls/) open-source web crawl. For the sake of brevity, the data record links among websites, not web pages. The information for nodes and links is the same as the toy example above.

In [6]:
# If you're running on a machine (e.g., Windows) that doesn't have wget or gzip,
# feel free to comment this out and use a different set of commands to load
# the data.
!wget https://khoury.northeastern.edu/home/dasmith/courses/cs6200/vertices-edu.txt.gz
!gzip -df vertices-edu.txt.gz
!wget https://khoury.northeastern.edu/home/dasmith/courses/cs6200/edges-edu.txt.gz
!gzip -df edges-edu.txt.gz

--2023-10-04 20:26:46--  https://khoury.northeastern.edu/home/dasmith/courses/cs6200/vertices-edu.txt.gz
Resolving khoury.northeastern.edu (khoury.northeastern.edu)... 52.70.229.197
Connecting to khoury.northeastern.edu (khoury.northeastern.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.khoury.northeastern.edu/home/dasmith/courses/cs6200/vertices-edu.txt.gz [following]
--2023-10-04 20:26:46--  https://www.khoury.northeastern.edu/home/dasmith/courses/cs6200/vertices-edu.txt.gz
Resolving www.khoury.northeastern.edu (www.khoury.northeastern.edu)... 52.70.229.197
Connecting to www.khoury.northeastern.edu (www.khoury.northeastern.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3703486 (3.5M) [application/x-gzip]
Saving to: ‘vertices-edu.txt.gz’

vertices-edu.txt.gz 100%[===================>]   3.53M  22.2MB/s    in 0.2s    

2023-10-04 20:26:46 (22.2 MB/s) - ‘vertices-edu.

There should now be files `vertices-edu.txt` and `edges-edu.txt`.

In [7]:
# TODO: Process the raw data into the same format as the simple graph.
# You may create lists or iterators.
# Open the file in read mode
edges = []
vertices = []
with open('edges-edu.txt', 'r') as edges_file:
    for line in edges_file:
      line = line.split()
      edges.append((int(line[0]), int(line[1])))

with open('vertices-edu.txt', 'r') as vertices_file:
    for line in vertices_file:
      line = line.split()
      vertices.append((int(line[0]), line[1]))

print(edges[:10])
print(vertices[:10])

[(386, 440), (19202, 1033), (103884, 2635), (342306, 7399), (8366, 8312), (8358, 8312), (8949, 8987), (8982, 8987), (8910, 8987), (9028, 8999)]
[(0, 'edu.00zl5e'), (1, 'edu.06hxbt'), (2, 'edu.082ifc'), (3, 'edu.083mjs'), (4, 'edu.09xzrr'), (5, 'edu.0aoqqj'), (6, 'edu.0ax4el'), (7, 'edu.0c5fez'), (8, 'edu.0cosn2'), (9, 'edu.0dcdp8')]


Refine your implementation of PageRank to test for numerical convergence. Specificially, at each iteration, calculate the [perplexity](https://en.wikipedia.org/wiki/Perplexity) of the PageRank distribution, where perplexity is defined as 2 raised to the [Shannon entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)) of the PageRank distribution, i.e., $2^{H(PR)}$. (Recall the definition of entropy from our discussion of data compression.) The maximum perplexity of a PageRank distribution will therefore be the number of nodes in the graph.

At each iteration, check the _change_ in perplexity. If the change is less than some threshold, you can stop.


In [8]:
from scipy import sparse

In [15]:
# TODO: Implement convergence testing in PageRank
# If you choose, you can share some subroutines with your first version.
# Print the change in perplexity at each iteration.

def page_rank(nodes, edges, threshold=1):
  N = len(nodes)
  d = 0.9
  cols, rows = zip(*edges)
  adj = sparse.csr_array((np.ones(len(edges)), (rows, cols)), shape=(N, N))
  r = sparse.csr_matrix(np.ones((N, 1)))
  r /= r.sum()
  perp_change = np.inf
  perplexity = 2**-(r.toarray()*np.log(r.toarray())).sum()
  adj /= adj.sum(axis=0)
  rand_jump = sparse.csr_array(np.ones((N, 1)) * (1 - d) / N)
  while perp_change >= threshold:
    print(perp_change)
    r = d * adj @ r
    r += rand_jump
    r /= r.sum()
    perplexity_new = 2**-(r.toarray()*np.log(r.toarray())).sum()
    perp_change = np.abs(perplexity - perplexity_new)
    perplexity = perplexity_new
  return [(nodes[i][0], nodes[i][1], r[i, 0]) for i in range(N)]

# Run until perplexity changes by less than 1
PR = page_rank(vertices, edges, 1)

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_base.py:659: RuntimeWarning: divide by zero encountered in divide
  recip = np.true_divide(1., other)


inf
5757.513642950361
239.98536303373749
214.77752859686825
3.1316528847069094
79.28615703364449
18.20889971670067
45.5071024209542
17.290024488838753
30.82950259862946
14.41621746227338
25.192047320637357
13.228470605353323
22.82108956264301
13.749020885477876
22.39783312857071
15.292342986790572
23.381425183930787
17.62250142586754
25.343556692148468
20.63164988741164
28.08224021084561
24.200206227275885
31.465377703405466
28.23854984038917
35.35207773883485
32.636200207595266
39.594701482656546
37.24874204402249
44.01233817706543
41.891970170449895
48.38525694756663
46.341799063395
52.463038222210116
50.34713298247834
55.97779802045966
53.650279802850264
58.67124769009956
56.015940645504315
60.32618714488967
57.26353213552193
60.79686643361515
57.295281983283985
60.03276658462141
56.112951666442314
58.0874856060891
53.81858020099935
55.11026253845148
50.59825453872918
51.32209087922445
46.69289199949344
46.98208793630272
42.363535404659274
42.35241284006395
37.85895803154699
37.6690

## Link Analysis

\[20 points\] In this final section, you will compute some properties of this web-site graph.

First, consider the _in-link count_ of a website, simply the number of websites pointing to it (including self-links).

In [28]:
# TODO: List the document ID, domain name, and in-link count of the 70 websites with the highest in-link count
N = len(vertices)
d = 0.9
cols, rows = zip(*edges)
adj = sparse.csr_array((np.ones(len(edges)), (rows, cols)), shape=(N, N))
in_link_counts = np.array(adj.sum(axis=1)).flatten()
website_info = [(vertices[i][0], vertices[i][1], in_link_counts[i]) for i in range(N)]
sorted_websites = sorted(website_info, key=lambda x: x[2], reverse=True)
sorted_websites[:70]

[(185524, 'edu.mit.web', 4388.0),
 (278032, 'edu.stanford', 4021.0),
 (244433, 'edu.purdue.english.owl', 3531.0),
 (140443, 'edu.indiana', 3339.0),
 (237176, 'edu.princeton', 3251.0),
 (64587, 'edu.columbia', 3123.0),
 (465503, 'edu.yale', 2804.0),
 (418623, 'edu.utexas', 2622.0),
 (383763, 'edu.unc', 2592.0),
 (197698, 'edu.nap', 2494.0),
 (439637, 'edu.washington', 2291.0),
 (373442, 'edu.umich', 2281.0),
 (440674, 'edu.washington.depts', 2276.0),
 (148945, 'edu.jhu.muse', 2255.0),
 (60975, 'edu.colorado', 2232.0),
 (449738, 'edu.wisc', 2230.0),
 (38320, 'edu.bu', 2205.0),
 (83572, 'edu.dartmouth', 1965.0),
 (408380, 'edu.usc', 1952.0),
 (178879, 'edu.mit', 1946.0),
 (27307, 'edu.berkeley', 1908.0),
 (233405, 'edu.pitt', 1857.0),
 (191069, 'edu.msu', 1810.0),
 (326371, 'edu.uchicago.press', 1763.0),
 (136464, 'edu.illinois', 1753.0),
 (93874, 'edu.educause', 1741.0),
 (56979, 'edu.cmu.cs', 1730.0),
 (199032, 'edu.ncsu', 1709.0),
 (36294, 'edu.brown', 1702.0),
 (202182, 'edu.nd', 1689

Then, use the PageRank values compute by your second implementation. Note that some websites will have both a high in-link count and PageRank.

In [20]:
# TODO: List the document ID, domain name, and PageRank of the 70 websites with the highest PageRank.
sorted(PR, key=lambda x: x[2], reverse=True)[:70]

[(319795, 'edu.ucdavis.ice.code', 0.10609002908518399),
 (318278, 'edu.ucdavis.cas', 0.08322150079378479),
 (210535, 'edu.northcentralcollege.brilliantfuture', 0.00929159438910175),
 (53834, 'edu.chop.securelogin', 0.008373300564430046),
 (53697, 'edu.chop.cidmfed', 0.007943070804735605),
 (210532, 'edu.northcentralcollege', 0.007421050003020955),
 (432977, 'edu.villanova.login', 0.007270018910749308),
 (432980, 'edu.villanova.lpres', 0.006942949137970304),
 (34517, 'edu.boisestate.cs-prod', 0.0068378992963070515),
 (34799, 'edu.boisestate.weblogin', 0.006686189945066094),
 (212726, 'edu.northwestern.rogersgroup', 0.004758176567074042),
 (138638, 'edu.illinois.matse.rogers', 0.00460628024572871),
 (457163, 'edu.wosc.jobs', 0.004290202196795437),
 (105087, 'edu.fsu.cpeip', 0.002933444637692426),
 (68629, 'edu.cord.moodle', 0.00287549787679982),
 (202000, 'edu.ncu', 0.002868672362180379),
 (68596, 'edu.cord.bprdeis', 0.0027660274722079428),
 (105088, 'edu.fsu.cpeipstore', 0.0027497417407

Finally, compute some summary statistics on this dataset.

In [52]:
# TODO: Compute:
# - the proportion of websites with no in-links (i.e., source nodes);
print(np.count_nonzero(np.array(adj.sum(axis=1) == 0)) / N)
# - the proportion of websites with no out-links (i.e., sink nodes);
print(np.count_nonzero(np.array(adj.sum(axis=0) == 0)) / N)
# - the proportion of websites whose PageRank is higher than the initial uniform distribution.
uniform_distribution = np.ones(N) / N
PR_values = [PR[i][2] for i in range(N)]
print(np.sum(PR_values > uniform_distribution) / N)

0.26153633041013563
0.6106641661427643
0.06337123189872879
